In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('KnownAlexanderProperties - Sheet1.csv',dtype='str')

## Data clean

In [3]:
df2 = df[['Display Properties','Known Properties']]

## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
df2 = df2.dropna()

In [6]:
df2

,Display Properties,Known Properties
0,"Lake Avenue, Miami Beach, FL","2135 Lake Avenue, Miami Beach, FL 33140"
1,14652 Biscayne Boulevard North Miami FL,14652 Biscayne Boulevard North Miami FL
2,"143 Nighthawk Avenue, Plantation, FL 33324","143 Nighthawk Avenue, Plantation, FL 33324"
3,"25 Northwest 34 Street Miami, FL","25 Northwest 34 Street Miami, FL"
4,"14600 Biscayne Boulevard, North Miami Beach, F...","14600 Biscayne Boulevard, North Miami Beach, F..."
5,"Bal Bay Drive, Bal Harbour, FL 33154-1313","248 Bal Bay Drive, Bal Harbour, FL 33154-1313"
6,"969 West 34th Street, Miami Beach, FL","969 West 34th Street, Miami Beach, FL"
7,"North Meridian Avenue, Miami Beach, FL","3451 North Meridian Avenue, Miami Beach, FL"
8,"2687 Flamingo Drive, Miami Beach, FL 33140","2687 Flamingo Drive, Miami Beach, FL 33140"
9,"5401 Collins Avenue, Unit 13-1 in Miami Beach ...","5401 Collins Avenue, Unit 13-1 in Miami Beach"


In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df2['geocoded'] = df2['Known Properties'].apply(geocode)

In [8]:
df2['geocoded'] = df2['geocoded'].astype(str)
df2[['lat', 'lon']] = df2['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df2['lat'] = df2['lat'].astype(float)
df2['lon'] = df2['lon'].astype(float)

In [9]:
df2.columns

Index(['Display Properties', 'Known Properties', 'geocoded', 'lat', 'lon'], dtype='object')

In [18]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap

df = df2

def tooltip_html(row_index, max_chars=30):
    """
    Generate HTML content for the tooltip with line breaks at word boundaries
    and without any labels.
    
    Parameters:
    - row_index (int): The index of the row in the DataFrame.
    - max_chars (int): Maximum number of characters per line.
    
    Returns:
    - str: HTML string for the tooltip content.
    """
    # Retrieve the address from 'Display Properties'
    address = df.at[row_index, 'Display Properties']
    address_str = str(address).strip() if pd.notnull(address) else 'N/A'
    
    # Use textwrap to wrap text without breaking words
    wrapped_address = '<br>'.join(textwrap.wrap(address_str, width=max_chars, break_long_words=False))
    
    # Construct the tooltip HTML without any labels
    tooltip_content = f'<div class="popup-content">{wrapped_address}</div>'
    return tooltip_content

# Initialize the map centered around the mean latitude and longitude
map_center = [25.781613207245726, -80.19350063432182]
m = folium.Map(location=map_center, zoom_start=12)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips
custom_css = """
<style>
    .popup-content {
        max-width: 200px; /* Adjusts the maximum width of the tooltip */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-tooltip {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""

# Add the custom CSS to the map
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Known Alexander Properties</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a CircleMarker without the 'icon' parameter
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=folium.Tooltip(tooltip_content, sticky=True),
        color='red',
        fill=True,
        fill_color='red',
        radius=9,
        weight=1,          # Thickness of the circle border
        opacity=1,       # Opacity of the circle border
        fill_opacity=1   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m

In [19]:
m.save('index.html')

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Alexander_Map
